# 🚀 Gemma-2-2b-it 改良プロンプト + QLoRA 訓練ノートブック

## 📋 概要
このノートブックは、**改良されたプロンプト**と**QLoRA (Quantized LoRA)** を使用して、MAP - Charting Student Math Misunderstandingsコンペティション用のGemma-2-2b-itモデルを効率的に訓練します。

### 🆕 主な改良点
1. **📝 改良プロンプト**: `final_compact_prompt.py`の最適化されたプロンプト構造
   - 65個の全ラベル完全対応（False_Correct:NA含む）
   - 分類ガイドラインの早期配置
   - 問題コンテキストの最適化
   
2. **⚡ QLoRA最適化**: メモリ効率とトレーニング速度の向上
   - 4-bit量子化による大幅なメモリ削減
   - 高速な勾配計算
   - GPU使用率の最適化

### 🎯 モデル仕様
- **ベースモデル**: google/gemma-2-2b-it (~2.6B parameters)
- **タスク**: 65ラベル分類 (Category:Misconception形式)
- **ファインチューニング**: QLoRA (Quantized Low-Rank Adaptation)
- **プロンプト**: final_compact_prompt.py の改良版
- **GPU**: T4/A100 対応

### 💾 保存場所
- **新モデル名**: `gemma-2-2b-improved-prompts-qlora`
- **Google Drive**: `/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models/`
- **圧縮ファイル**: zipダウンロード対応

## 📁 環境セットアップ

In [ ]:
# Google Driveマウント
from google.colab import drive
drive.mount('/content/drive')

# 改良版モデル用ディレクトリ作成
import os
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/trained_models', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/map_data', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/logs', exist_ok=True)
os.makedirs('/content/drive/MyDrive/kaggleCompe_MAP-math/improved_models', exist_ok=True)

print("✅ Google Drive マウント完了")
print("✅ ディレクトリ作成完了")
print("📁 改良版モデル保存用ディレクトリ: /content/drive/MyDrive/kaggleCompe_MAP-math/improved_models/")

In [ ]:
# Hugging Face認証
!huggingface-cli login

In [ ]:
# QLoRA訓練用ライブラリのインストール
!pip install peft accelerate transformers datasets bitsandbytes -U -q
!pip install scipy -U -q  # QLoRAに必要

print("✅ QLoRA対応ライブラリインストール完了")
print("📦 インストール済み: transformers, peft, accelerate, bitsandbytes, datasets")

## 📚 ライブラリ読み込みと設定

In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    AutoConfig,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    BitsAndBytesConfig
)
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import warnings
import json
import shutil
from pathlib import Path

warnings.filterwarnings("ignore")

# 改良版モデル設定
MODEL_NAME = "google/gemma-2-2b-it"
NUM_LABELS = 65  # 全ラベル対応
MAX_LENGTH = 512
NEW_MODEL_NAME = "gemma-2-2b-improved-prompts-qlora"

# デバイス設定
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🖥️ 使用デバイス: {device}")

if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 GPU メモリ: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")

print(f"✅ ライブラリ読み込み完了")
print(f"🚀 新モデル名: {NEW_MODEL_NAME}")
print(f"📊 対応ラベル数: {NUM_LABELS}")
print(f"📏 最大トークン長: {MAX_LENGTH}")

## 📝 改良プロンプト関数の実装

In [ ]:
def get_actual_labels_from_data(train_df):
    """実際のデータから全ラベルを取得（Colab用に修正）"""
    labels = (
        train_df["Category"].astype(str)
        + ":"
        + train_df["Misconception"].fillna("NA").astype(str)
    )
    return sorted(labels.unique())

def get_improved_compact_prompt(question, answer, explanation, all_labels):
    """改良版コンパクトプロンプト - final_compact_prompt.pyベース"""
    
    labels_text = "\n".join([f"- {label}" for label in all_labels])
    
    prompt = f"""You are an expert math educator analyzing student responses for mathematical misconceptions.

Question: {question}
Correct Answer: {answer}
Student's Explanation: {explanation}

CLASSIFICATION GUIDELINES:
• True_Correct:NA = Student demonstrates correct understanding
• False_Correct:NA = Student gives correct answer but for wrong reasons  
• True_Neither:NA = Correct answer but unclear/incomplete reasoning
• False_Neither:NA = Incorrect answer but no specific misconception identified
• True_Misconception:[Type] = Correct answer but demonstrates specific misconception
• False_Misconception:[Type] = Incorrect answer with identifiable misconception

TASK: Classify this student's response using EXACTLY ONE of these {len(all_labels)} labels:

{labels_text}

Classification:"""
    
    return prompt

def create_enhanced_text_with_improved_prompt(row, all_labels):
    """改良プロンプトを使用したテキスト特徴量作成"""
    question = str(row["QuestionText"]) if pd.notna(row["QuestionText"]) else ""
    mc_answer = str(row["MC_Answer"]) if pd.notna(row["MC_Answer"]) else ""
    explanation = str(row["StudentExplanation"]) if pd.notna(row["StudentExplanation"]) else ""
    
    # 改良されたプロンプト形式を使用
    enhanced_text = get_improved_compact_prompt(question, mc_answer, explanation, all_labels)
    return enhanced_text

print("✅ 改良プロンプト関数実装完了")
print("📝 final_compact_prompt.py の改良版プロンプトを使用")
print("🎯 特徴: 分類ガイドライン早期配置、65ラベル完全対応")

## 📊 データ読み込みと改良前処理

In [ ]:
def load_and_prepare_improved_data():
    """改良プロンプトを使用したデータ読み込みと前処理"""
    print("=" * 60)
    print("📊 改良プロンプト版 データ読み込みと前処理")
    print("=" * 60)

    # データパス（Google Drive内）
    train_data_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/map_data/train.csv"

    if not os.path.exists(train_data_path):
        print(f"❌ データが見つかりません: {train_data_path}")
        print("💡 Google DriveのMyDrive/kaggleCompe_MAP-math/map_data/に train.csv をアップロードしてください")
        return None, None

    try:
        # データ読み込み
        train_df = pd.read_csv(train_data_path)
        print(f"✅ 訓練データ読み込み: {train_df.shape}")

        # NaN値除去
        before_len = len(train_df)
        train_df = train_df.dropna(subset=["Category", "StudentExplanation", "Misconception"])
        after_len = len(train_df)
        print(f"🧹 NaN除去: {before_len} -> {after_len} ({before_len - after_len}行削除)")

        # 全ラベルの取得（65個）
        all_labels = get_actual_labels_from_data(train_df)
        print(f"📋 全ラベル数: {len(all_labels)}")
        
        # 主要ラベルの確認
        print("🎯 主要ラベル例:")
        for i, label in enumerate(all_labels[:5]):
            count = (train_df["Category"].astype(str) + ":" + train_df["Misconception"].fillna("NA").astype(str) == label).sum()
            print(f"  {i+1}. {label} ({count}件)")
        print(f"  ... ({len(all_labels)-5}個の追加ラベル)")

        # False_Correct:NAの確認
        false_correct_present = "False_Correct:NA" in all_labels
        print(f"✅ False_Correct:NA含有: {false_correct_present}")

        # 改良プロンプトを使用したテキスト特徴量作成
        print("\n🔧 改良プロンプトによるテキスト特徴量作成中...")
        train_df["enhanced_text"] = train_df.apply(
            lambda row: create_enhanced_text_with_improved_prompt(row, all_labels), 
            axis=1
        )

        # ラベル作成（Category:Misconception形式）
        train_df["full_label"] = (
            train_df["Category"].astype(str) 
            + ":" 
            + train_df["Misconception"].fillna("NA").astype(str)
        )

        # テキスト長統計
        text_lengths = train_df["enhanced_text"].str.len()
        print(f"\n📝 改良プロンプトのテキスト長統計:")
        print(f"  平均: {text_lengths.mean():.1f} 文字")
        print(f"  中央値: {text_lengths.median():.1f} 文字")
        print(f"  最大: {text_lengths.max()} 文字")
        print(f"  推定トークン数（平均）: {text_lengths.mean() / 4:.0f} トークン")
        
        # 長いプロンプトの割合
        long_prompts = (text_lengths > 2048).sum()
        print(f"  2048文字超: {long_prompts} ({long_prompts/len(train_df)*100:.1f}%)")

        return train_df, all_labels

    except Exception as e:
        print(f"❌ データ読み込みまたは前処理に失敗: {e}")
        return None, None

# データ読み込み実行
train_df, all_labels = load_and_prepare_improved_data()

## 🤖 QLoRA対応Gemmaモデル読み込み

In [ ]:
def load_gemma_model_with_qlora():
    """QLoRA設定でGemma-2-2b-itモデルとトークナイザーを読み込み"""
    print("\n" + "=" * 60)
    print(f"🤖 QLoRA対応Gemmaモデル読み込み: {MODEL_NAME}")
    print("=" * 60)

    try:
        # 🔧 QLoRA用量子化設定
        bnb_config = BitsAndBytesConfig(
            load_in_4bit=True,                      # 4-bit量子化を有効
            bnb_4bit_use_double_quant=True,         # ダブル量子化で精度向上
            bnb_4bit_quant_type="nf4",              # Normal Float 4-bit
            bnb_4bit_compute_dtype=torch.float16,   # 計算精度
        )
        
        print("⚡ QLoRA量子化設定:")
        print("  🔸 4-bit量子化: 有効")
        print("  🔸 ダブル量子化: 有効")  
        print("  🔸 量子化タイプ: nf4")
        print("  🔸 計算精度: float16")

        # 📝 トークナイザー読み込み
        print("\n📝 Gemmaトークナイザー読み込み中...")
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        
        if tokenizer.pad_token is None:
            tokenizer.pad_token = tokenizer.eos_token
            print("🔧 パディングトークンをEOSトークンに設定")

        print(f"✅ トークナイザー読み込み完了")
        print(f"🔖 パディングトークン: {tokenizer.pad_token}")
        print(f"📏 語彙サイズ: {tokenizer.vocab_size:,}")

        # 🧠 QLoRA対応モデル読み込み
        print(f"\n🧠 QLoRA対応Gemmaモデル読み込み中...")
        
        config = AutoConfig.from_pretrained(MODEL_NAME)
        config.num_labels = NUM_LABELS
        config.problem_type = "single_label_classification"
        
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME,
            config=config,
            quantization_config=bnb_config,  # QLoRA量子化設定
            device_map="auto",               # 自動デバイス配置
            trust_remote_code=True,
        )

        print(f"✅ QLoRA対応Gemmaモデル読み込み完了!")

        # 📊 モデル情報表示
        total_params = sum(p.numel() for p in model.parameters())
        print(f"\n📊 モデル情報:")
        print(f"  📈 総パラメータ数: {total_params:,}")
        print(f"  📊 分類クラス数: {NUM_LABELS}")
        print(f"  💡 モデルサイズ: ~{total_params / 1e9:.2f}B parameters")
        print(f"  ⚡ 量子化: 4-bit (メモリ使用量約75%削減)")

        return model, tokenizer

    except Exception as e:
        print(f"❌ QLoRAモデル読み込み失敗: {e}")
        return None, None

# QLoRA対応モデル読み込み実行
if train_df is not None:
    model, tokenizer = load_gemma_model_with_qlora()
else:
    print("❌ データが読み込まれていないため、モデル読み込みをスキップします")
    model, tokenizer = None, None

## ⚡ QLoRA設定とPEFT適用

In [ ]:
def apply_qlora_to_model(model):
    """QLoRA設定とPEFT適用（メモリ効率化）"""
    print("\n" + "=" * 60)
    print("⚡ QLoRA (PEFT) 設定と適用")
    print("=" * 60)

    if model is None:
        print("❌ ベースモデルが準備されていません")
        return None

    try:
        import peft
        print(f"📦 PEFTライブラリバージョン: {peft.__version__}")

        # 🔧 量子化モデルをQLoRA訓練用に準備
        print("🔄 量子化モデルをQLoRA訓練用に準備中...")
        model = prepare_model_for_kbit_training(model)
        print("✅ 量子化モデル準備完了")

        # ⚡ QLoRA設定（メモリ効率重視）
        qlora_config = LoraConfig(
            r=16,                           # LoRA attention dimension  
            lora_alpha=32,                  # Alpha parameter for LoRA scaling
            target_modules=[               # Gemma-2の主要な線形層をターゲット
                "q_proj", "k_proj", "v_proj", "o_proj",
                "gate_proj", "up_proj", "down_proj"
            ],
            lora_dropout=0.05,             # Dropout probability for LoRA layers
            bias="none",                   # Bias type
            task_type="SEQ_CLS",           # Sequence Classification task
            modules_to_save=["classifier", "score"],  # 分類ヘッドを保存
        )

        print("📋 QLoRA設定:")
        print(f"  🔸 r (rank): {qlora_config.r}")
        print(f"  🔸 lora_alpha: {qlora_config.lora_alpha}")
        print(f"  🔸 target_modules: {qlora_config.target_modules}")
        print(f"  🔸 lora_dropout: {qlora_config.lora_dropout}")
        print(f"  🔸 task_type: {qlora_config.task_type}")
        print(f"  🔸 modules_to_save: {qlora_config.modules_to_save}")

        # 🚀 QLoRAモデル作成
        print("\n🔄 QLoRAモデル作成中...")
        qlora_model = get_peft_model(model, qlora_config)

        print("✅ QLoRAモデル作成完了")
        qlora_model.print_trainable_parameters()

        # 📊 メモリ使用量確認
        if torch.cuda.is_available():
            memory_used = torch.cuda.memory_allocated() / 1e9
            memory_total = torch.cuda.get_device_properties(0).total_memory / 1e9
            print(f"\n💾 GPU メモリ使用状況:")
            print(f"  使用中: {memory_used:.2f} GB")
            print(f"  総容量: {memory_total:.2f} GB")
            print(f"  使用率: {memory_used/memory_total*100:.1f}%")

        return qlora_model

    except Exception as e:
        print(f"❌ QLoRA適用失敗: {e}")
        return None

# QLoRA適用
if model is not None:
    qlora_model = apply_qlora_to_model(model)
else:
    print("❌ モデルが読み込まれていないため、QLoRAを適用できません")
    qlora_model = None

## 📝 改良プロンプト版データセット作成

In [ ]:
class ImprovedMathMisconceptionDataset(Dataset):
    """改良プロンプト版 Math Misconception Dataset for PyTorch"""

    def __init__(self, texts, labels, tokenizer, max_length=512):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]

        # 改良プロンプトのトークン化（長いテキスト対応）
        encoding = self.tokenizer(
            text,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )

        return {
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }

def compute_improved_map3_metrics(eval_pred):
    """改良版MAP@3メトリクスの計算"""
    predictions, labels = eval_pred
    predictions = torch.softmax(torch.tensor(predictions), dim=-1).numpy()

    map_scores = []
    for i, true_label in enumerate(labels):
        top3_indices = np.argsort(predictions[i])[::-1][:3]
        score = 0.0
        for j, pred_idx in enumerate(top3_indices):
            if pred_idx == true_label:
                score = 1.0 / (j + 1)
                break
        map_scores.append(score)

    map3_score = np.mean(map_scores)
    accuracy = accuracy_score(labels, np.argmax(predictions, axis=1))

    return {
        "map3": map3_score, 
        "accuracy": accuracy,
        "map3_detailed": f"MAP@3: {map3_score:.4f}"
    }

print("✅ 改良プロンプト版データセットクラス定義完了")
print("📊 MAP@3評価メトリクス（改良版）定義完了")
print("🎯 特徴: 長いプロンプト対応、詳細メトリクス")

## 🚀 QLoRA最適化訓練実行

In [ ]:
def train_improved_qlora_model(train_df, qlora_model, tokenizer, all_labels):
    """改良プロンプト + QLoRA モデルのファインチューニング実行"""
    print("\n" + "=" * 60)
    print("🚀 改良プロンプト + QLoRA ファインチューニング開始")
    print("=" * 60)

    if train_df is None or qlora_model is None:
        print("❌ 訓練データまたはモデルが準備されていません")
        return None, None

    # 65ラベル対応のラベルエンコーディング
    label_encoder = LabelEncoder()
    train_df["encoded_labels"] = label_encoder.fit_transform(train_df["full_label"])

    print(f"📋 改良プロンプト版ラベル情報:")
    print(f"  総ラベル数: {len(label_encoder.classes_)}")
    print(f"  エンコード例（最初の5個）:")
    for i, label in enumerate(label_encoder.classes_[:5]):
        count = (train_df["encoded_labels"] == i).sum()
        print(f"    {i}: {label} ({count:,}件)")
    print(f"    ... ({len(label_encoder.classes_)-5}個の追加ラベル)")

    # 訓練/検証分割
    train_texts = train_df["enhanced_text"].tolist()
    train_labels = train_df["encoded_labels"].tolist()

    try:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts,
            train_labels,
            test_size=0.2,
            random_state=42,
            stratify=train_labels,
        )
        print("✅ Stratified split適用")
    except ValueError:
        X_train, X_val, y_train, y_val = train_test_split(
            train_texts, train_labels, test_size=0.2, random_state=42
        )
        print("✅ Regular split適用（少数クラスのため）")

    print(f"📊 改良プロンプト版データ分割:")
    print(f"  訓練データ: {len(X_train):,}件")
    print(f"  検証データ: {len(X_val):,}件")

    # 改良プロンプト版データセット作成
    train_dataset = ImprovedMathMisconceptionDataset(
        X_train, y_train, tokenizer, max_length=MAX_LENGTH
    )
    val_dataset = ImprovedMathMisconceptionDataset(
        X_val, y_val, tokenizer, max_length=MAX_LENGTH
    )

    # QLoRA最適化訓練設定
    training_args = TrainingArguments(
        output_dir=f"/content/drive/MyDrive/kaggleCompe_MAP-math/improved_models/{NEW_MODEL_NAME}",
        num_train_epochs=3,                    # 改良プロンプト用最適エポック数
        per_device_train_batch_size=2,         # QLoRA用小バッチサイズ
        per_device_eval_batch_size=2,
        gradient_accumulation_steps=16,        # 実効バッチサイズ32を維持
        warmup_steps=200,
        weight_decay=0.01,
        logging_dir=f"/content/drive/MyDrive/kaggleCompe_MAP-math/logs/{NEW_MODEL_NAME}",
        logging_steps=50,                      # より頻繁なログ
        eval_strategy="steps",
        eval_steps=200,                        # より頻繁な評価
        save_strategy="steps",
        save_steps=200,
        load_best_model_at_end=True,
        metric_for_best_model="map3",
        greater_is_better=True,
        report_to="none",
        dataloader_pin_memory=False,
        fp16=False,                           # QLoRAでは無効
        bf16=True,                            # QLoRA推奨設定
        optim="adamw_torch",
        learning_rate=1e-4,                   # QLoRA用学習率
        save_total_limit=3,
        gradient_checkpointing=True,          # メモリ効率化
        dataloader_num_workers=0,             # QLoRA安定化
        group_by_length=True,                 # 効率的バッチング
    )

    print("\n📋 QLoRA最適化訓練設定:")
    print(f"  🔸 エポック数: {training_args.num_train_epochs}")
    print(f"  🔸 per_device_batch_size: {training_args.per_device_train_batch_size}")
    print(f"  🔸 gradient_accumulation: {training_args.gradient_accumulation_steps}")
    print(f"  🔸 実効バッチサイズ: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
    print(f"  🔸 学習率: {training_args.learning_rate}")
    print(f"  🔸 保存先: {training_args.output_dir}")
    print(f"  🔸 gradient_checkpointing: {training_args.gradient_checkpointing}")
    print(f"  🔸 group_by_length: {training_args.group_by_length}")

    # QLoRA対応Trainer作成
    trainer = Trainer(
        model=qlora_model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_improved_map3_metrics,
    )

    # 訓練実行
    print("\n🚀 改良プロンプト + QLoRA ファインチューニング開始...")
    try:
        trainer.train()
        print("✅ ファインチューニング完了")

        # 最終評価
        print("\n📊 最終評価:")
        eval_results = trainer.evaluate()
        for key, value in eval_results.items():
            if isinstance(value, float):
                print(f"  🎯 {key}: {value:.4f}")
            else:
                print(f"  📋 {key}: {value}")

        return trainer, label_encoder

    except Exception as e:
        print(f"❌ ファインチューニング中にエラー: {e}")
        return None, None

# 改良プロンプト + QLoRA 訓練実行
if train_df is not None and qlora_model is not None and all_labels is not None:
    trainer, label_encoder = train_improved_qlora_model(train_df, qlora_model, tokenizer, all_labels)
else:
    print("❌ 訓練に必要な要素が準備されていません")
    trainer, label_encoder = None, None

## 💾 改良版モデル保存

In [ ]:
def save_improved_qlora_model(trainer, tokenizer, label_encoder, all_labels):
    """改良プロンプト + QLoRA 訓練済みモデルの保存"""
    print("\n" + "=" * 60)
    print("💾 改良プロンプト + QLoRA モデル保存")
    print("=" * 60)

    if trainer is None or tokenizer is None or label_encoder is None:
        print("❌ 保存に必要な要素が準備されていません")
        return False

    save_base_path = "/content/drive/MyDrive/kaggleCompe_MAP-math/improved_models"
    model_save_path = f"{save_base_path}/{NEW_MODEL_NAME}-final"

    try:
        # モデルとトークナイザーの保存
        print(f"📁 保存先: {model_save_path}")

        # QLoRAモデル保存（アダプタのみ）
        trainer.save_model(model_save_path)
        tokenizer.save_pretrained(model_save_path)

        print("✅ 改良プロンプト + QLoRAモデル保存完了")

        # 改良版ラベルマッピングの保存
        label_mapping = {i: label for i, label in enumerate(label_encoder.classes_)}
        label_file = f"{model_save_path}/improved_label_mapping.json"

        with open(label_file, "w", encoding="utf-8") as f:
            json.dump(label_mapping, f, ensure_ascii=False, indent=2)

        print("✅ 改良版ラベルマッピング保存完了")
        print(f"📋 ラベル総数: {len(label_mapping)}")
        
        # False_Correct:NAの確認
        false_correct_labels = [label for label in label_mapping.values() if "False_Correct" in label]
        print(f"✅ False_Correct関連ラベル: {len(false_correct_labels)}個含有")

        # 改良版メタデータの保存
        metadata = {
            "model_name": MODEL_NAME,
            "improved_model_name": NEW_MODEL_NAME,
            "num_labels": len(all_labels),
            "max_length": MAX_LENGTH,
            "task_type": "sequence_classification",
            "architecture": "AutoModelForSequenceClassification",
            "improvements": {
                "prompt_optimization": "final_compact_prompt.py based",
                "qlora_applied": True,
                "quantization": "4-bit",
                "label_coverage": "65 labels including False_Correct:NA",
                "prompt_features": [
                    "Early classification guidelines placement",
                    "Enhanced context structure", 
                    "Comprehensive label coverage",
                    "Optimized token efficiency"
                ]
            },
            "qlora_config": {
                "r": 16,
                "lora_alpha": 32,
                "target_modules": ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
                "task_type": "SEQ_CLS",
                "quantization": "4-bit nf4",
                "compute_dtype": "float16"
            },
            "training_info": {
                "epochs": 3,
                "batch_size": 2,
                "gradient_accumulation": 16,
                "learning_rate": 1e-4,
                "optimization": "QLoRA + improved prompts"
            }
        }

        metadata_file = f"{model_save_path}/improved_model_metadata.json"
        with open(metadata_file, "w", encoding="utf-8") as f:
            json.dump(metadata, f, ensure_ascii=False, indent=2)

        print("✅ 改良版メタデータ保存完了")

        # 保存されたファイルの確認
        print(f"\n📂 保存されたファイル:")
        for item in os.listdir(model_save_path):
            item_path = os.path.join(model_save_path, item)
            if os.path.isfile(item_path):
                size_mb = os.path.getsize(item_path) / (1024 * 1024)
                print(f"  📄 {item}: {size_mb:.2f} MB")
            else:
                print(f"  📁 {item}/")

        return True

    except Exception as e:
        print(f"❌ モデル保存中にエラー: {e}")
        return False

# 改良版モデル保存実行
if trainer is not None and tokenizer is not None and label_encoder is not None and all_labels is not None:
    save_success = save_improved_qlora_model(trainer, tokenizer, label_encoder, all_labels)

    if save_success:
        print("\n🎉 改良プロンプト + QLoRAモデル保存完了!")
        print(f"📁 Google Drive保存パス: /content/drive/MyDrive/kaggleCompe_MAP-math/improved_models/")
        print(f"🚀 新モデル名: {NEW_MODEL_NAME}")
        print("🔥 次のステップ: Kaggleデータセットとしてアップロード準備完了")
    else:
        print("❌ モデル保存に失敗しました")
else:
    print("❌ 保存に必要な要素が準備されていません")

## 📦 改良版モデルzip作成とダウンロード

In [ ]:
def create_improved_model_zip():
    """改良プロンプト + QLoRAモデル用zipファイル作成"""
    print("\n" + "=" * 60)
    print("📦 改良版モデル用zipファイル作成")
    print("=" * 60)

    try:
        import zipfile

        model_dir = f"/content/drive/MyDrive/kaggleCompe_MAP-math/improved_models/{NEW_MODEL_NAME}-final"
        zip_path = f"/content/drive/MyDrive/kaggleCompe_MAP-math/{NEW_MODEL_NAME}.zip"

        if not os.path.exists(model_dir):
            print(f"❌ モデルディレクトリが見つかりません: {model_dir}")
            return False

        print(f"📁 ソース: {model_dir}")
        print(f"📦 出力zip: {zip_path}")

        with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
            for root, dirs, files in os.walk(model_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, model_dir)
                    zipf.write(file_path, arcname)
                    print(f"  ➕ {arcname}")

        # zipファイルサイズ確認
        zip_size_mb = os.path.getsize(zip_path) / (1024 * 1024)
        print(f"\n✅ 改良版モデルzip作成完了!")
        print(f"📏 zipサイズ: {zip_size_mb:.2f} MB")
        print(f"📥 ダウンロードパス: {zip_path}")

        # ダウンロード用コード表示
        print(f"\n💡 Colabでダウンロードするには:")
        print(f"```python")
        print(f"from google.colab import files")
        print(f"files.download('{zip_path}')")
        print(f"```")

        return True

    except Exception as e:
        print(f"❌ zip作成中にエラー: {e}")
        return False

# 改良版モデルzip作成実行
zip_success = create_improved_model_zip()

# 自動ダウンロード
if zip_success:
    print("\n🎉 改良プロンプト + QLoRAモデルzip作成完了!")
    print(f"📦 zipファイル: {NEW_MODEL_NAME}.zip")
    
    # ダウンロード確認
    try:
        from google.colab import files
        response = input("\n改良版モデルを今すぐダウンロードしますか？ (y/n): ")
        if response.lower() == 'y':
            files.download(f'/content/drive/MyDrive/kaggleCompe_MAP-math/{NEW_MODEL_NAME}.zip')
            print("✅ ダウンロード開始!")
        else:
            print("💡 後でダウンロードする場合は上記のコードを使用してください")
    except:
        print("💡 手動でダウンロードしてください")
else:
    print("❌ zip作成に失敗しました")

## 🎯 改良版モデル完了サマリー

### ✅ 完了した改良点
1. **📝 プロンプト改良**: `final_compact_prompt.py`ベース
   - 65個の全ラベル完全対応（False_Correct:NA含む）
   - 分類ガイドラインの早期配置
   - 問題コンテキストの最適化構造
   - トークン効率の改善

2. **⚡ QLoRA最適化**: メモリ効率と性能向上
   - 4-bit量子化による大幅なメモリ削減
   - LoRA (r=16, alpha=32) による効率的ファインチューニング
   - 勾配チェックポイントによるメモリ最適化
   - バッチサイズとaccumulationの最適バランス

3. **🚀 訓練効率化**:
   - QLoRA対応の専用データセット作成
   - MAP@3メトリクス改良版実装
   - グルーピングバッチングによる効率化
   - メモリ使用量の大幅削減

### 📁 保存されたファイル
- **新モデル名**: `gemma-2-2b-improved-prompts-qlora`
- **Google Drive**: `/content/drive/MyDrive/kaggleCompe_MAP-math/improved_models/`
- **zip形式**: `gemma-2-2b-improved-prompts-qlora.zip`
- **メタデータ**: 改良内容の詳細記録付き

### 🔄 次のステップ（Kaggle提出準備）
1. **Kaggleデータセット作成**: 改良版モデルをKaggleにアップロード
2. **推論テスト**: test.csvでの推論性能確認
3. **submission.csv生成**: MAP@3最適化による提出
4. **性能比較**: 元モデルとの性能差分析

### 🎯 改良版モデルの読み込み方法（Kaggle用）
```python
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from peft import PeftModel

# ベースモデル読み込み
base_model = AutoModelForSequenceClassification.from_pretrained("google/gemma-2-2b-it")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-2-2b-it")

# QLoRAアダプタ読み込み
model = PeftModel.from_pretrained(base_model, "path/to/improved_model")
```

### 🚀 主な改善効果期待値
- **メモリ使用量**: 約75%削減（QLoRA効果）
- **プロンプト精度**: False_Correct:NA対応による分類精度向上
- **訓練効率**: 改良プロンプトによる学習収束性向上
- **推論速度**: 最適化されたトークン長による高速化

**🎉 改良プロンプト + QLoRAモデル訓練完了!**
**次は提出用推論ノートブックでの性能テストです！**